# <b> Naive Bayes </b>
___

<b> Table of Content: </b>
<br> [Pipeline_1](#100)
<br> [Pipeline 2](#200)
<br> [Pipeline 3](#300)

Loading Modules and Datasets

In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [12]:
# read csv file to a pandas dataframe
df_pipeline1 = pd.read_csv("pipeline_1.csv")

<a id = "100"> <h2> Pipeline 1 </h2> </a>
___

> Declare Features and Target

In [6]:
# show all columns in dataset
#list(df_pipeline1.columns)[:]

In [7]:
# ---------------------------------------------------------------------------------------------------
# Extract needed features as a list of columns 
# excluding target variable 'Revenue' and original object features (that were labelencoded), namely 'Month' and VisitorType'
# ---------------------------------------------------------------------------------------------------
## create a list of all the columns
list_all_columns = df_pipeline1.columns.tolist()

## remove features in list, which are not needed for testing for feature selection, namely:
list_remove_features = ['Revenue'] 

# Create sets of a,b
setA = set(list_all_columns)
setB = set(list_remove_features)

# Get new set with elements that are only in a but not in b
setlist_X_columns = setA.difference(list_remove_features)

# convert set object to a list
list_X_columns = list(setlist_X_columns)

# Define dependent variables
X = df_pipeline1[list_X_columns].to_numpy()

#show the columns that need to be included as features
## sort alphabetically
list_X_columns.sort()
## show list 
#list_X_columns

In [8]:
# Define Features and Target variables
X = df_pipeline1[list_X_columns]
Y = df_pipeline1['Revenue']

# Split dataset into training set and test set 
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=2019) 

In [9]:
# Create a Gaussian Classifier
gnb = GaussianNB()

# Train the model using the training sets
gnb.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [19]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy Naive Bayes, pipeline 1:", metrics.accuracy_score(y_test, y_pred).round(4))

Accuracy Naive Bayes, pipeline 1: 0.3601


<a id = "200"> <h2> Pipeline 2 </h2> </a>
___

<a id = "300"> <h2> Pipeline 3 </h2> </a>
___